In [105]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [106]:
modeldata = pd.read_csv("/mnt/data/big_csv_files/output_csv_full.csv",low_memory=False) #load data

In [107]:
pd.set_option('future.no_silent_downcasting', True)

In [108]:
print(modeldata.head())
print(modeldata.tail())

   time_ref  account code country_code product_type        value status
0    202412  Exports   00           AD        Goods       2581.0      F
1    202412  Exports   00           AE        Goods  323384662.0      F
2    202412  Exports   00           AG        Goods     266255.0      F
3    202412  Exports   00           AI        Goods      11760.0      F
4    202412  Exports   00           AL        Goods     639168.0      F
         time_ref  account    code country_code product_type     value status
2456285    201406  Imports  A12123           US     Services  168141.0      F
2456286    201406  Imports  A12123           VN     Services  112094.0      F
2456287    201406  Imports  A12123           VU     Services   56047.0      F
2456288    201406  Imports  A12123           WS     Services   56047.0      F
2456289    201406  Imports  A12123           ZA     Services   56047.0      F


In [109]:
modeldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2456290 entries, 0 to 2456289
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   time_ref      int64  
 1   account       object 
 2   code          object 
 3   country_code  object 
 4   product_type  object 
 5   value         float64
 6   status        object 
dtypes: float64(1), int64(1), object(5)
memory usage: 131.2+ MB


In [110]:
print(f"Dataset contains {modeldata.shape[0]} rows and {modeldata.shape[1]} columns.\n")

Dataset contains 2456290 rows and 7 columns.



In [111]:
modeldata.describe()

,time_ref,value
count,2.456290e+06,2.304323e+06
mean,2.019189e+05,5.096921e+06
std,3.096674e+02,1.655224e+08
min,2.014060e+05,-1.616938e+09
25%,2.016120e+05,2.384000e+03
50%,2.019090e+05,2.205900e+04
75%,2.022060e+05,1.966490e+05
max,2.024120e+05,2.409993e+10


In [112]:
print(modeldata['account'].describe()) #2 unique values. Can be hot coded

count     2456290
unique          2
top       Imports
freq      1400991
Name: account, dtype: object


In [115]:
modeldata_encoded = pd.get_dummies(modeldata, columns=['account'], drop_first=True) #encode account one hot
modeldata = pd.concat([modeldata, modeldata_encoded], axis=1) # merge unencoded and encoded dataframe
modeldata

,time_ref,account,code,country_code,product_type,value,status,time_ref,code,country_code,...,status,time_ref,code,country_code,product_type,value,status,account_Imports,account_Imports,account_Imports
0,202412,Exports,00,AD,Goods,2581.0,F,202412,00,AD,...,F,202412,00,AD,Goods,2581.0,F,False,False,False
1,202412,Exports,00,AE,Goods,323384662.0,F,202412,00,AE,...,F,202412,00,AE,Goods,323384662.0,F,False,False,False
2,202412,Exports,00,AG,Goods,266255.0,F,202412,00,AG,...,F,202412,00,AG,Goods,266255.0,F,False,False,False
3,202412,Exports,00,AI,Goods,11760.0,F,202412,00,AI,...,F,202412,00,AI,Goods,11760.0,F,False,False,False
4,202412,Exports,00,AL,Goods,639168.0,F,202412,00,AL,...,F,202412,00,AL,Goods,639168.0,F,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2456285,201406,Imports,A12123,US,Services,168141.0,F,201406,A12123,US,...,F,201406,A12123,US,Services,168141.0,F,True,True,True
2456286,201406,Imports,A12123,VN,Services,112094.0,F,201406,A12123,VN,...,F,201406,A12123,VN,Services,112094.0,F,True,True,True
2456287,201406,Imports,A12123,VU,Services,56047.0,F,201406,A12123,VU,...,F,201406,A12123,VU,Services,56047.0,F,True,True,True
2456288,201406,Imports,A12123,WS,Services,56047.0,F,201406,A12123,WS,...,F,201406,A12123,WS,Services,56047.0,F,True,True,True


In [116]:
print(modeldata['code'].describe()) #1407 unique values. Cannot be hotcoded

           code     code     code     code     code     code     code     code
count   2456290  2456290  2456290  2456290  2456290  2456290  2456290  2456290
unique     1407     1407     1407     1407     1407     1407     1407     1407
top          00       00       00       00       00       00       00       00
freq      17915    17915    17915    17915    17915    17915    17915    17915


In [117]:
print(modeldata['country_code'].describe()) #251 unique values. Cannot be hotcoded

       country_code country_code country_code country_code country_code  \
count       2454712      2454712      2454712      2454712      2454712   
unique          251          251          251          251          251   
top              AU           AU           AU           AU           AU   
freq          93778        93778        93778        93778        93778   

       country_code country_code country_code  
count       2454712      2454712      2454712  
unique          251          251          251  
top              AU           AU           AU  
freq          93778        93778        93778  


In [118]:
print(modeldata['product_type'].describe()) #2 unique values. Can be hot coded

       product_type product_type product_type product_type product_type  \
count       2456290      2456290      2456290      2456290      2456290   
unique            2            2            2            2            2   
top           Goods        Goods        Goods        Goods        Goods   
freq        2179965      2179965      2179965      2179965      2179965   

       product_type product_type product_type  
count       2456290      2456290      2456290  
unique            2            2            2  
top           Goods        Goods        Goods  
freq        2179965      2179965      2179965  


In [119]:
modeldata_encoded = pd.get_dummies(modeldata, columns=['product_type'], drop_first=True) #encode product type one hot
modeldata = pd.concat([modeldata, modeldata_encoded], axis=1) # merge unencoded and encoded dataframe
modeldata

,time_ref,account,code,country_code,product_type,value,status,time_ref,code,country_code,...,account_Imports,account_Imports,product_type_Services,product_type_Services,product_type_Services,product_type_Services,product_type_Services,product_type_Services,product_type_Services,product_type_Services
0,202412,Exports,00,AD,Goods,2581.0,F,202412,00,AD,...,False,False,False,False,False,False,False,False,False,False
1,202412,Exports,00,AE,Goods,323384662.0,F,202412,00,AE,...,False,False,False,False,False,False,False,False,False,False
2,202412,Exports,00,AG,Goods,266255.0,F,202412,00,AG,...,False,False,False,False,False,False,False,False,False,False
3,202412,Exports,00,AI,Goods,11760.0,F,202412,00,AI,...,False,False,False,False,False,False,False,False,False,False
4,202412,Exports,00,AL,Goods,639168.0,F,202412,00,AL,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2456285,201406,Imports,A12123,US,Services,168141.0,F,201406,A12123,US,...,True,True,True,True,True,True,True,True,True,True
2456286,201406,Imports,A12123,VN,Services,112094.0,F,201406,A12123,VN,...,True,True,True,True,True,True,True,True,True,True
2456287,201406,Imports,A12123,VU,Services,56047.0,F,201406,A12123,VU,...,True,True,True,True,True,True,True,True,True,True
2456288,201406,Imports,A12123,WS,Services,56047.0,F,201406,A12123,WS,...,True,True,True,True,True,True,True,True,True,True


In [120]:
print(modeldata['status'].describe()) #3 unique values. Can be hot coded

         status   status   status   status   status   status   status  \
count   2456290  2456290  2456290  2456290  2456290  2456290  2456290   
unique        3        3        3        3        3        3        3   
top           F        F        F        F        F        F        F   
freq    2282345  2282345  2282345  2282345  2282345  2282345  2282345   

         status   status   status   status   status   status   status  \
count   2456290  2456290  2456290  2456290  2456290  2456290  2456290   
unique        3        3        3        3        3        3        3   
top           F        F        F        F        F        F        F   
freq    2282345  2282345  2282345  2282345  2282345  2282345  2282345   

         status   status  
count   2456290  2456290  
unique        3        3  
top           F        F  
freq    2282345  2282345  


In [121]:
modeldata_encoded = pd.get_dummies(modeldata, columns=['status'], drop_first=True) #encode account one hot
modeldata = pd.concat([modeldata, modeldata_encoded], axis=1) # merge unencoded and encoded dataframe
modeldata

,time_ref,account,code,country_code,product_type,value,status,time_ref,code,country_code,...,status_F,status_R,status_F,status_R,status_F,status_R,status_F,status_R,status_F,status_R
0,202412,Exports,00,AD,Goods,2581.0,F,202412,00,AD,...,True,False,True,False,True,False,True,False,True,False
1,202412,Exports,00,AE,Goods,323384662.0,F,202412,00,AE,...,True,False,True,False,True,False,True,False,True,False
2,202412,Exports,00,AG,Goods,266255.0,F,202412,00,AG,...,True,False,True,False,True,False,True,False,True,False
3,202412,Exports,00,AI,Goods,11760.0,F,202412,00,AI,...,True,False,True,False,True,False,True,False,True,False
4,202412,Exports,00,AL,Goods,639168.0,F,202412,00,AL,...,True,False,True,False,True,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2456285,201406,Imports,A12123,US,Services,168141.0,F,201406,A12123,US,...,True,False,True,False,True,False,True,False,True,False
2456286,201406,Imports,A12123,VN,Services,112094.0,F,201406,A12123,VN,...,True,False,True,False,True,False,True,False,True,False
2456287,201406,Imports,A12123,VU,Services,56047.0,F,201406,A12123,VU,...,True,False,True,False,True,False,True,False,True,False
2456288,201406,Imports,A12123,WS,Services,56047.0,F,201406,A12123,WS,...,True,False,True,False,True,False,True,False,True,False


In [95]:
print(modeldata.isnull().sum())

time_ref             0
account              0
code                 0
country_code      1578
product_type         0
value           151967
status               0
dtype: int64


In [96]:
print(modeldata['value'].mean())

5096921.459327577


In [97]:
modeldata['value'] = modeldata['value'].fillna(modeldata['value'].mean())

In [98]:
print(modeldata.isnull())

         time_ref  account   code  country_code  product_type  value  status
0           False    False  False         False         False  False   False
1           False    False  False         False         False  False   False
2           False    False  False         False         False  False   False
3           False    False  False         False         False  False   False
4           False    False  False         False         False  False   False
...           ...      ...    ...           ...           ...    ...     ...
2456285     False    False  False         False         False  False   False
2456286     False    False  False         False         False  False   False
2456287     False    False  False         False         False  False   False
2456288     False    False  False         False         False  False   False
2456289     False    False  False         False         False  False   False

[2456290 rows x 7 columns]


In [99]:
print(modeldata.isnull().sum())

time_ref           0
account            0
code               0
country_code    1578
product_type       0
value              0
status             0
dtype: int64


In [104]:
modeldata

,time_ref,account,code,country_code,product_type,value,status,time_ref,code,country_code,...,product_type_Services,product_type_Services,status_F,status_R,status_F,status_R,status_F,status_R,status_F,status_R
0,202412,Exports,00,AD,Goods,2581.0,F,202412,00,AD,...,False,False,True,False,True,False,True,False,True,False
1,202412,Exports,00,AE,Goods,323384662.0,F,202412,00,AE,...,False,False,True,False,True,False,True,False,True,False
2,202412,Exports,00,AG,Goods,266255.0,F,202412,00,AG,...,False,False,True,False,True,False,True,False,True,False
3,202412,Exports,00,AI,Goods,11760.0,F,202412,00,AI,...,False,False,True,False,True,False,True,False,True,False
4,202412,Exports,00,AL,Goods,639168.0,F,202412,00,AL,...,False,False,True,False,True,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2456285,201406,Imports,A12123,US,Services,168141.0,F,201406,A12123,US,...,True,True,True,False,True,False,True,False,True,False
2456286,201406,Imports,A12123,VN,Services,112094.0,F,201406,A12123,VN,...,True,True,True,False,True,False,True,False,True,False
2456287,201406,Imports,A12123,VU,Services,56047.0,F,201406,A12123,VU,...,True,True,True,False,True,False,True,False,True,False
2456288,201406,Imports,A12123,WS,Services,56047.0,F,201406,A12123,WS,...,True,True,True,False,True,False,True,False,True,False
